## Notebook to test that the ramp and fitter objects work as expected

In [ ]:
import pandas as pd
import numpy as np
from ramp_utils.ramp import RampTimeSeq,RampMeasurement
from ramp_utils.fitter import IterativeFitter
import matplotlib.pyplot as plt
import time, sys, pickle, bz2
from multiprocessing import Pool

%matplotlib notebook

### Test the ramp object itself

A Generic ramp with a user-specified time sequence

In [ ]:
dt,nf,ns,ng = 10.8,8,12,10
myramp = RampTimeSeq('GENERIC',ng,nframes=nf, nskips=ns, read_times=dt*np.arange(ng*(nf+ns)))
myramp.test_plot()

A WFC3 ramp with 15 samples and a SPARS25 sequence 

In [ ]:
myramp = RampTimeSeq('HST/WFC3/IR',15,samp_seq='SPARS25')
myramp.test_plot()

### Test the fitter on a single ramp

#### Choose the type of ramp

In [ ]:
#dt,nf,ns,ng = 10,2,10,10
#myramp = RampTimeSeq('GENERIC',ng,nframes=nf, nskips=ns, read_times=dt*np.arange(ng*(nf+ns)))  #  a generic ramp with the above sequence

myramp = RampTimeSeq('HST/WFC3/IR',15,samp_seq='SPARS100') # a WFC3 ramp with 15 samples and a SPARS100 sequence 

#### Choose the "true" flux (in e/s), and specify a desired sequence of Cosmic Rays impacts

In [ ]:
myflux = 64

#CRdict = {'times':[620., 80.,400.,980],'counts':[300,500,250,290]}
CRdict = None  # For no cosmic rays

#### Create a measurement.   
If the selected ramp is of **GENERIC** type, you need to specify gain, RON, KTC, bias and full_well.

In [ ]:
#mymeas = RampMeasurement(myramp,myflux,gain=1,RON=20,KTC=50,bias=10000,full_well=100000,CRdict=CRdict)   # an example of generic case use
mymeas = RampMeasurement(myramp,myflux,CRdict=CRdict)  #an example for WFC3 ramps

#### Run the fitter and print and display some diagnostics

In [ ]:
myramp.test_plot()

mymeas.test_plot()

myfitter = IterativeFitter(mymeas,fitpars={'one_iteration_method':'Nelder-Mead'})
err, count, gi, crlcount = myfitter.perform_fit(CRthr=4)
myfitter.test_plot()

print('Error message',err)
print('Iterations',count)
print('fitted flux (e/s)', myfitter.mean_electron_rate)
print('real flux (e/s)',myfitter.RM.flux)
print(gi)
print('Iterations for CR rej',crlcount)


### Goodness of fit measures

In [ ]:
myfitter.goodness_of_fit(mode='G-test')
print(myfitter.gof_stat,myfitter.gof_pval)

myfitter.goodness_of_fit(mode='Pearson-chi-sq')
print(myfitter.gof_stat,myfitter.gof_pval)

myfitter.goodness_of_fit(mode='Squared-deviations-nocov')
print(myfitter.gof_stat,myfitter.gof_pval)

myfitter.goodness_of_fit(mode='Squared-deviations')
print(myfitter.gof_stat,myfitter.gof_pval)
